In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from typing import Tuple
import math
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from deepant_pytorch import create_sinusoidal_time_series, check_cuda, normalize_data
from deepant_pytorch import split_data, calculate_size, SimpleTSDataset, DeepAnt

In [ ]:
func = create_sinusoidal_time_series
total_length = 100000
dev = torch.device(check_cuda())
series = func(total_length)

window = 45
dimensions = 1
prediction_window = 1
batch_size = 32
learning_rate = 0.001
epochs = 10
test_train_split = 0.8

In [ ]:
train, test = split_data(series, train_percent_size=test_train_split)

In [ ]:
train

In [ ]:
train.size, test.size

In [ ]:
train_ds = SimpleTSDataset(data=train, sequence_size=window, prediction_size=prediction_window, device=dev)
test_ds = SimpleTSDataset(data=test, sequence_size=window, prediction_size=prediction_window, device=dev)

In [ ]:
train.size, train_ds.max_range

In [ ]:
test.size, test_ds.max_range

In [ ]:
SimpleTSDataset??

In [ ]:
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

In [ ]:
model = DeepAnt(window=window, p_window=prediction_window, dimensions=dimensions)

In [ ]:
DeepAnt??

In [ ]:
criterion = nn.L1Loss()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
losses = list()

In [ ]:
for epoch in range(epochs):
    count = 0
    total_loss = 0
    for x, y in train_dl:
        model.train()
        x = x.requires_grad_()

        optimizer.zero_grad()
        result = model(x)
        loss = criterion(result, y)
        loss.backward()
        optimizer.step()
        current_loss = loss.item()
        total_loss += current_loss
        count += 1

    losses.append(total_loss / count)

In [ ]:
%matplotlib inline
plt.plot(np.array(losses))
plt.title("Training Loss")
plt.show()

In [ ]:
torch.save(model.state_dict, './deepant.pth')

In [ ]:
results = []
model.eval()
for x, y in test_dl:
    result = model(x)
    results.append(result)

In [ ]:
reconstruct = []
for data in results:
    data = data[:, 0, :]
    res = data.reshape(-1)
    res = res.detach().numpy()
    for r in res:
        reconstruct.append(r)

plt.plot(np.array(reconstruct))
plt.title("Output on test data")
plt.show()